[View in Colaboratory](https://colab.research.google.com/github/yeboda96/Charlottesville-Albemarle-Connectivity-Survey-Analysis/blob/master/CNN_Car_Classification.ipynb)

In [0]:
import scipy.io
import shutil
import os

In [22]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from keras.preprocessing import image
from keras.applications import vgg16
from keras.applications import vgg19
from keras.applications import resnet50
from keras.applications import inception_v3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Model,Sequential
from keras import optimizers
from keras.callbacks import TensorBoard, ModelCheckpoint
import argparse
from time import time

from skimage import exposure, color

from keras import backend as K
K.set_image_dim_ordering('tf')

In [0]:
def init_model(train_dir,val_dir,batch_size=16,model_name='vgg16',num_class=2,img_size=224):
    """
    initialize cnn model and training and validation data generator
    parms:
        args: parsed commandline arguments
    return:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
    """
    

    print('loading the model and the pre-trained weights...')

    # load base model
    if model_name == 'vgg16':
        base_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape = (224,224,3)) # need specify input_shape
        # this preprocess_input is the default preprocess func for given network, you can change it or implement your own 
        # use inception_v3 preprocess for vgg16, it seems that it works better than vgg16.preprocess_input
        preprocess_input = inception_v3.preprocess_input 

    # initalize training image data generator
    # you can also specify data augmentation here
    train_datagen = image.ImageDataGenerator(
        # width_shift_range=0.1,
        # height_shift_range=0.1,
        # samplewise_center=True,
        # samplewise_std_normalization=True,
        # rescale=1./255,
        preprocessing_function=preprocess_input, # preprocess_input,
        # rotation_range=30,
        # shear_range=0.1,
        # zoom_range=0.1,
        # vertical_flip=True,
        horizontal_flip=True
        )

    # initalize validation image data generator
    # you can also specify data augmentation here
    validation_datagen = image.ImageDataGenerator(
        # samplewise_center=True,
        # samplewise_std_normalization=True
        # rescale=1./255
        preprocessing_function=preprocess_input # preprocess_input
        )

    train_generator = train_datagen.flow_from_directory(
        train_dir,
        # color_mode='grayscale', # 'rgb'
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

    validation_generator = validation_datagen.flow_from_directory(
        val_dir,
        # color_mode='grayscale',  # 'rgb'
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical')

    # fix base_model layers
    for layer in base_model.layers:
        layer.trainable = False

    # added some customized layers for your own data
    x = base_model.output
    if model_name == 'vgg16':
        x = Flatten(name='flatten')(x)
        x = Dense(512, activation='relu', name='fc1-pretrain')(x)
        x = Dense(256, activation='relu', name='fc2-pretrain')(x)
        x = Dropout(0.5, name='dropout')(x)

    # added softmax layer
    predictions = Dense(num_class, activation='softmax', name='predictions')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(), metrics=["accuracy"])

    return model, train_generator, validation_generator

In [0]:
def train(model, train_generator, validation_generator, num_class= 2,model_name='vgg16', batch_size=16, epochs=30, suffix='laioffer'):
    """
    train the model
    parms:
        model: initialized model
        train_generator: training data generator
        validation_generator: validation data generator
        args: parsed command line arguments
    return:
    """
    # define number of steps/iterators per epoch
    stepsPerEpoch = train_generator.samples / batch_size
    validationSteps= validation_generator.samples / batch_size

    # save the snapshot of the model to local drive
    pretrain_model_name = 'pretrained_{}_{}_{}_{}.h5'.format(model_name, num_class, epochs, suffix)
    # visualize the training process
    tensorboard = TensorBoard(log_dir="logs/{}_pretrain_{}".format(model_name, time()), histogram_freq=0, write_graph=True)
    checkpoint = ModelCheckpoint(pretrain_model_name, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    callbacks_list = [checkpoint, tensorboard]

    model.fit_generator(
        train_generator,
        steps_per_epoch=stepsPerEpoch,
        epochs=epochs,
        callbacks = callbacks_list,
        validation_data = validation_generator,
        validation_steps=validationSteps)
    return model

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from google.colab import auth 
auth.authenticate_user()

In [35]:
# Generate creds for the Drive FUSE library.

from oauth2client.client import GoogleCredentials 
creds = GoogleCredentials.get_application_default()

import getpass

!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL

vcode = getpass.getpass()

!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
# Create a directory and mount Google Drive using that directory.

!mkdir -p drive

!google-drive-ocamlfuse drive

In [37]:
print ('Files in Drive:')

!ls drive/

Files in Drive:
 545_Team5_PP.doc
 545_Team5_PP.doc.odt
 car_devkit.tgz
 cars_test
 cars_test_annos_withlabels.mat
 cars_train
'Charlottesville _ Albemarle Connectivity Survey.desktop'
'Colab Notebooks'
'CREATE TABLE Train.docx'
 Data_Science_1.03.pdf
'How to get started with Drive'
'MGMT 382'
'MGMT 451 Presentation - Bar chart 1.ods'
'MGMT 451 Presentation - Column chart 1.ods'
'MGMT 451 Presentation - Column chart 3.ods'
'MGMT 451 Presentation - Line chart 1.ods'
'MGMT 451 Presentation - Line chart 2.ods'
'MGMT 451 Presentation - Line chart 3.ods'
'mgmt 472'
'MGMT 488.odt'
 MGMT_488_yelp
'Project 1 Plan.docx'
'Project 1 Plan.docx.odt'
 project2
'project 2.odt'
'project 2.pdf'
 UCA
'Untitled document.odt'
'Untitled document.odt (19b934c1)'
'Untitled presentation.pdf'


In [41]:
model, train_generator, validation_generator = init_model('drive/cars_train', 'drive/cars_test')
train(model, train_generator, validation_generator)

loading the model and the pre-trained weights...
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Epoch 1/30


AttributeError: ignored